In [2]:
from pyspark.sql import SparkSession, HiveContext
from pyspark.sql.types import *
import pyspark.sql.functions as F
import os

spark = SparkSession \
    .builder \
    .appName("xml parser") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "10g") \
    .config("spark.executor.instances","10") \
    .enableHiveSupport()\
    .getOrCreate()

In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks.spark.xml:spark-xml_2.11-0.9.0 pyspark-shell'

In [4]:
df = spark.read.format("com.databricks.spark.xml").option("rowTag","Root").load("file:///home/cm2506/Documents/data/customer.xml")

In [5]:
df.printSchema()

root
 |-- Customers: struct (nullable = true)
 |    |-- Customer: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- CompanyName: string (nullable = true)
 |    |    |    |-- ContactName: string (nullable = true)
 |    |    |    |-- ContactTitle: string (nullable = true)
 |    |    |    |-- Fax: string (nullable = true)
 |    |    |    |-- FullAddress: struct (nullable = true)
 |    |    |    |    |-- Address: string (nullable = true)
 |    |    |    |    |-- City: string (nullable = true)
 |    |    |    |    |-- Country: string (nullable = true)
 |    |    |    |    |-- PostalCode: long (nullable = true)
 |    |    |    |    |-- Region: string (nullable = true)
 |    |    |    |-- Phone: string (nullable = true)
 |    |    |    |-- _CustomerID: string (nullable = true)
 |-- Orders: struct (nullable = true)
 |    |-- Order: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- CustomerID: string (

In [5]:
df.count()

1

In [7]:
field_info=[]
field_info.append(1)
def function_to_flatten_json(input_df):
    flat_columns = [c[0] for c in input_df.dtypes if c[1][:6] != 'struct' and c[1][:5] !="array"]
    print("flat_columns : ",flat_columns )
    nested_columns = [c[0] for c in input_df.dtypes if c[1][:6] == 'struct']
    array_columns=[c[0] for c in input_df.dtypes if c[1][:5] == 'array']
    print("array_columns",array_columns)
    print("nested_columns :",nested_columns)
    for ac in array_columns:
        input_df=input_df.select(flat_columns+nested_columns+[x for x in array_columns if x != ac ]+[F.explode(ac).alias(ac)])
        print(input_df.columns)
    input_df = input_df.select(flat_columns +array_columns+
                               [F.col(nested_column+'.'+c).alias(nested_column+'_'+c)
                                for nested_column in nested_columns
                                for c in input_df.select(nested_column+'.*').columns])
    field_info.pop()
    field_info.append(input_df)
    print("input_dataframe ",",".join([x[1][:6] for x in input_df.dtypes]))
    new_nested=[c[0] for c in input_df.dtypes if c[1][:6] == 'struct']
    new_array=[c[0] for c in input_df.dtypes if c[1][:5] == 'array']
    if new_nested or new_array :        
        function_to_flatten_json(input_df)
function_to_flatten_json(df)
field_info_df=field_info[0]

flat_columns :  []
array_columns []
nested_columns : ['Customers', 'Orders']
input_dataframe  array<,array<
flat_columns :  []
array_columns ['Customers_Customer', 'Orders_Order']
nested_columns : []
['Orders_Order', 'Customers_Customer']
['Customers_Customer', 'Orders_Order']
input_dataframe  struct,struct
flat_columns :  []
array_columns []
nested_columns : ['Customers_Customer', 'Orders_Order']
input_dataframe  string,string,string,string,struct,string,string,string,bigint,string,string,struct
flat_columns :  ['Customers_Customer_CompanyName', 'Customers_Customer_ContactName', 'Customers_Customer_ContactTitle', 'Customers_Customer_Fax', 'Customers_Customer_Phone', 'Customers_Customer__CustomerID', 'Orders_Order_CustomerID', 'Orders_Order_EmployeeID', 'Orders_Order_OrderDate', 'Orders_Order_RequiredDate']
array_columns []
nested_columns : ['Customers_Customer_FullAddress', 'Orders_Order_ShipInfo']
input_dataframe  string,string,string,string,string,string,string,bigint,string,string,

In [8]:
field_info_df.printSchema()

root
 |-- Customers_Customer_CompanyName: string (nullable = true)
 |-- Customers_Customer_ContactName: string (nullable = true)
 |-- Customers_Customer_ContactTitle: string (nullable = true)
 |-- Customers_Customer_Fax: string (nullable = true)
 |-- Customers_Customer_Phone: string (nullable = true)
 |-- Customers_Customer__CustomerID: string (nullable = true)
 |-- Orders_Order_CustomerID: string (nullable = true)
 |-- Orders_Order_EmployeeID: long (nullable = true)
 |-- Orders_Order_OrderDate: string (nullable = true)
 |-- Orders_Order_RequiredDate: string (nullable = true)
 |-- Customers_Customer_FullAddress_Address: string (nullable = true)
 |-- Customers_Customer_FullAddress_City: string (nullable = true)
 |-- Customers_Customer_FullAddress_Country: string (nullable = true)
 |-- Customers_Customer_FullAddress_PostalCode: long (nullable = true)
 |-- Customers_Customer_FullAddress_Region: string (nullable = true)
 |-- Orders_Order_ShipInfo_Freight: double (nullable = true)
 |-- Orde

In [6]:
result=field_info_df.write.option("header","true").csv('customer')

NameError: name 'field_info_df' is not defined